In [2]:
import sys
import re
import pandas as pd
df_sample = pd.read_csv('dample.csv')
#print(df_sample[['EID','Email Address','Home Phone','Address1', 'City', 'State', 'Zip Code']])

# -------- Email Diff ---------
df_ph_csv = df_sample[['EID','Home Phone']]
df_ph_csv = df_ph_csv[df_ph_csv['EID'].notna()] # drops only rows where eid column is na
#df_ph_csv = df_ph_csv.dropna() #drops all rows that 'na' in any columns
#adding empty column
df_ph_csv['telephone'] = ''
print(df_ph_csv)
raw_eid = df_ph_csv['EID'].dropna()
eid = raw_eid.tolist()
#print(eid)
for i in range(len(df_ph_csv)) : 
    #df_ph_csv.iloc[i,2]=re.findall(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]',df_ph_csv.iloc[i,1])
    if pd.isnull(df_ph_csv.iloc[i,1]):
        continue
    df_ph_csv.iloc[i,2]=df_ph_csv.iloc[i,1].replace("(","").replace(")","").replace("-","").replace("/","")
    print(df_ph_csv.iloc[i,2])
print(df_ph_csv)

          EID      Home Phone telephone
0      SPEEDY   (512)699-4971          
1       GWA99  (512) 585-4209          
3     ADAMSDI   (936)520-1117          
4      SLA583   (214)498-3495          
5    0847MNCD   (513)843-4010          
..        ...             ...       ...
993    WCW363   (512)940-8980          
994  WARNERPT             NaN          
995  WARNERPT             NaN          
996  0495TEWA   (512)795-8995          
997  0684QHGX   (281)658-2235          

[994 rows x 3 columns]
5126994971
512 5854209
9365201117
2144983495
5138434010
3184581505
7138235611
5126574880
2146740564
8329692928
8173073742
3618156260
2104169922
5122930468
5125600450
8322746058
8322746058
5126899957
7136222270
3038757629
7135157126
2146839583
8174738848
7132568888
7132568888
5126367537
5127125366
5124510723
5129262330
5129262330
5127959263
7137030111
5125518277
2813810078
5125358429
2106325307
2819352945
7132481833
2108287797
2107716534
5122172443
5125896090
2818088848
9564599000
8302295423


9727099627
7134444497
2143948388
7138700387
9147559484
5129928300
4325531145
5127516738
5124315717
5124703957
9174883116
9174883116
5124523268
5126335739
7135421238
8324741555
8329959245
2142630083
8179940988
2108635914
7137803824
7137803824
6309476407
5129252266
5123273700
5123940557
3126140198
2816515592
2547182081
2547182081
5126328832
2148023480
5127395501
5127395501
5129234335
5129403989
737 7089385
5127508122
2148240794
9565332493
7138283590
5124784844
5124221633
5124221633
8325309019
8172353008
5129131700
5129131700
5124744742
2813439602
9793242513
2147242822
7139531010
7138584517
2145223300
2812295633
2148506593
7132544333
9727576725
5127795775
5122664773
9792775196
5126881818
2143532789
5126897650
7136614853
5124965545
8305967687
5128691630
5128691630
7133922852
5122805419
5122805419
7138267748
2814351907
2125186676
5136732171
5124231779
5124231779
5124369717
5127951833
8325525832
7137816699
7137816699
8308993258
5124152804
5129750114
8302780257
8326594082
5123311717
512329562

In [3]:
import cx_Oracle
connection = cx_Oracle.connect("vipapp/ZJf3__6qjRt8@exa1-scan.austin.utexas.edu:1525/viprp_pdb.austin.utexas.edu")
cursor = connection.cursor()
cursor.execute("select UT_EID, vi_phone_area_code, vi_phone_nbr from vi_vips_main m, vi_vips_phone_info p where m.ALUMNI_ID = p.ALUMNI_ID AND m.ut_eid IN %s" % str(tuple(eid)).replace(',)',')'))
results = cursor.fetchall()
df_ph_db = results
cursor.close()
connection.close()
print(df_ph_db)
df_ph_db = pd.DataFrame(results, columns = ['EID','Area','phone'])
#dropping duplicates
df_ph_db = df_ph_db.drop_duplicates()
print(df_ph_db)

[('0223RZWI', '361', '4632443'), ('0227VECQ', '956', '3305027'), ('0228CVDP', '512', '7316755'), ('0254CMBM', '512', '3760773'), ('0254CMBM', '830', '6726518'), ('0267SSAH', '512', '5076093'), ('0282AANF', '512', '4312768'), ('0277SKBB', '512', '9373521'), ('0252RFCH', '254', '6984861'), ('0252RFCH', '254', '6981590'), ('0282XJNC', '312', '5135300'), ('0286JMED', '214', '6362682'), ('0289IHVS', '281', '9804552'), ('0884UVNI', '704', '6050343'), ('0335RZJQ', '512', '5548990'), ('0495TEWA', '512', '7958995'), ('0332IYDE', '817', '9057773'), ('0834NNSF', '956', '8673332'), ('0333SCXZ', '512', '4979104'), ('0544FKIJ', '254', '7607759'), ('0339MRJN', '214', '2320078'), ('0342MUJH', '760', '6686453'), ('0344UWHZ', '512', '5896090'), ('0652ADSS', '512', '4849351'), ('0345DGHM', '512', '4316677'), ('2558VRUR', '512', '5928914'), ('0444UEXV', '941', '5045599'), ('0884TJIW', '512', '9220467'), ('0345ZEGA', '559', '2877341'), ('0345ZEGA', '559', '6792292'), ('0353PQTU', '940', '7816048'), ('0376B

In [4]:
df_ph_db["telephone"] = df_ph_db["Area"].astype(str) + df_ph_db["phone"].astype(str)
#df_ph_db["telephone"] = df_ph_db["Area"] + df_ph_db["phone"]
print(df_ph_db)
#print(df_ph_db.iloc[:,3])

           EID Area    phone   telephone
0     0223RZWI  361  4632443  3614632443
1     0227VECQ  956  3305027  9563305027
2     0228CVDP  512  7316755  5127316755
3     0254CMBM  512  3760773  5123760773
4     0254CMBM  830  6726518  8306726518
...        ...  ...      ...         ...
2381    WSH375  512  6956346  5126956346
2383      WSM3  214  3511331  2143511331
2384    WWS426  512  5773000  5125773000
2385    WWS426  512  2944513  5122944513
2386    WWS426  125  5773000  1255773000

[2183 rows x 4 columns]


In [5]:
# the phone number columns are string type and they are being converted to numeric 
#df_ph_db[['Area', 'phone','telephone']] = df_ph_db[['Area', 'phone','telephone']].apply(pd.to_numeric)
#print(df_ph_db.dtypes)

ValueError: Unable to parse string "6849990EXT203" at position 465

In [25]:
df_phone = pd.DataFrame()
#print(df_email)
for i in range(len(df_ph_csv)):
    id_csv = df_ph_csv.iloc[i,0]
    ph_csv = df_ph_csv.iloc[i,2]
    ph_db = (df_ph_db.loc[df_ph_db['EID'] == id_csv]).iloc[:,3]
    ph_db = {*ph_db}
    #print(ph_db, ph_csv)
    ph_csv = {ph_csv}
    diff = (ph_csv - ph_db)
    if len(diff) > 0:
            df_phone = df_phone.append({'EID':id_csv, 'telephone':df_ph_csv.iloc[i,2]}, ignore_index=True)

df_phone = df_phone.drop_duplicates()
#print(df_phone)
#df_phone.to_csv(r'C:\Users\le5752\Documents\DB_Python\ph.csv', index = False)

df_phone['DB_phone'] = ''
for i in range(len(df_phone)):
    id_csv = df_phone.iloc[i,0]
    ph_db = (df_ph_db.loc[df_ph_db['EID'] == id_csv]).iloc[:,3]
    #print(ph_db)
    df_phone.iloc[i,2] = ph_db.values.tolist()
    
df_phone = df_phone[df_phone['telephone'].notna()] 
df_phone['telephone'].dropna()
print(df_phone['telephone'].dropna())
#df_phone.to_csv(r'C:\Users\le5752\Documents\DB_Python\ph.csv', index = False)


0      512 5854209
1       3184581505
2       2146740564
3       5122930468
4       5126899957
          ...     
385     8172472483
386               
387               
388               
389               
Name: telephone, Length: 358, dtype: object
